In [1]:
from pyspark.sql import SparkSession
from ucimlrepo import fetch_ucirepo
from pyspark.sql import functions as F
from pyspark.sql import Window
import matplotlib.pyplot as plt
import seaborn as sns
import os
print(f'{os.getcwd()}')
import pandas as pd

/home/jdwolfe/Spark-Projects/spark-learning-notebooks


# We will find the pitcher's who gave given up the most amount of one inning cycles

In [2]:
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

25/05/11 13:37:57 WARN Utils: Your hostname, JordansGaming resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/11 13:37:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/11 13:37:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [30]:
csv_headers = [
    "game id", "visiting team", "inning", "batting team", "outs", "balls", "strikes",
    "pitch sequence", "vis score", "home score", "batter", "batter hand", "res batter",
    "res batter hand", "pitcher", "pitcher hand", "res pitcher", "res pitcher hand",
    "catcher", "first base", "second base", "third base", "shortstop", "left field",
    "center field", "right field", "first runner", "second runner", "third runner",
    "event text", "leadoff flag", "pinchhit flag", "defensive position", "lineup position",
    "event type", "batter event flag", "ab flag", "hit value", "SH flag", "SF flag",
    "outs on play", "double play flag", "triple play flag", "RBI on play", "wild pitch flag",
    "passed ball flag", "fielded by", "batted ball type", "bunt flag", "foul flag",
    "hit location", "num errors", "1st error player", "1st error type", "2nd error player",
    "2nd error type", "3rd error player", "3rd error type", "batter dest",
    "runner on 1st dest", "runner on 2nd dest", "runner on 3rd dest", "play on batter",
    "play on runner on 1st", "play on runner on 2nd", "play on runner on 3rd",
    "SB for runner on 1st flag", "SB for runner on 2nd flag", "SB for runner on 3rd flag",
    "CS for runner on 1st flag", "CS for runner on 2nd flag", "CS for runner on 3rd flag",
    "PO for runner on 1st flag", "PO for runner on 2nd flag", "PO for runner on 3rd flag",
    "Responsible pitcher for runner on 1st", "Responsible pitcher for runner on 2nd",
    "Responsible pitcher for runner on 3rd", "New Game Flag", "End Game Flag",
    "Pinch-runner on 1st", "Pinch-runner on 2nd", "Pinch-runner on 3rd",
    "Runner removed for pinch-runner on 1st", "Runner removed for pinch-runner on 2nd",
    "Runner removed for pinch-runner on 3rd", "Batter removed for pinch-hitter",
    "Position of batter removed for pinch-hitter", "Fielder with First Putout",
    "Fielder with Second Putout", "Fielder with Third Putout", "Fielder with First Assist",
    "Fielder with Second Assist", "Fielder with Third Assist", "Fielder with Fourth Assist",
    "Fielder with Fifth Assist", "event num"
]

full_df = None

for file in os.listdir(f'{os.getcwd()}/data/mlb_data/decade_pbp_files'):
    file_path = f'{os.getcwd()}/data/mlb_data/decade_pbp_files/{file}'
    print(f'Processing {file_path}')
    df = spark.read.csv(file_path, header=False, inferSchema=True).toDF(*csv_headers)

    if full_df is None:
        full_df = df
    else:
        full_df = full_df.unionByName(df)

full_df.show()


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/decade_pbp_files/full_1910s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/decade_pbp_files/full_1970s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/decade_pbp_files/full_1930s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/decade_pbp_files/full_2000s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/decade_pbp_files/full_1960s_pbp.csv


...

Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/decade_pbp_files/full_1980s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/decade_pbp_files/full_1990s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/decade_pbp_files/full_2020s_pbp.csv


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/decade_pbp_files/full_2010s_pbp.csv


+------------+-------------+------+------------+----+-----+-------+--------------+---------+----------+--------+-----------+----------+---------------+--------+------------+-----------+----------------+--------+----------+-----------+----------+---------+----------+------------+-----------+------------+-------------+------------+---------------+------------+-------------+------------------+---------------+----------+-----------------+-------+---------+-------+-------+------------+----------------+----------------+-----------+---------------+----------------+----------+----------------+---------+---------+------------+----------+----------------+--------------+----------------+--------------+----------------+--------------+-----------+------------------+------------------+------------------+--------------+---------------------+---------------------+---------------------+-------------------------+-------------------------+-------------------------+-------------------------+--------------

In [4]:
full_df.printSchema()

root
 |-- game id: string (nullable = true)
 |-- visiting team: string (nullable = true)
 |-- inning: integer (nullable = true)
 |-- batting team: integer (nullable = true)
 |-- outs: integer (nullable = true)
 |-- balls: integer (nullable = true)
 |-- strikes: integer (nullable = true)
 |-- pitch sequence: string (nullable = true)
 |-- vis score: integer (nullable = true)
 |-- home score: integer (nullable = true)
 |-- batter: string (nullable = true)
 |-- batter hand: string (nullable = true)
 |-- res batter: string (nullable = true)
 |-- res batter hand: string (nullable = true)
 |-- pitcher: string (nullable = true)
 |-- pitcher hand: string (nullable = true)
 |-- res pitcher: string (nullable = true)
 |-- res pitcher hand: string (nullable = true)
 |-- catcher: string (nullable = true)
 |-- first base: string (nullable = true)
 |-- second base: string (nullable = true)
 |-- third base: string (nullable = true)
 |-- shortstop: string (nullable = true)
 |-- left field: string (nulla

In [31]:
full_df.count()

15217681

In [32]:
full_df = full_df.withColumn('Season', F.substring(F.col('game id'), 4, 4).cast('int'))

full_df.show()

+------------+-------------+------+------------+----+-----+-------+--------------+---------+----------+--------+-----------+----------+---------------+--------+------------+-----------+----------------+--------+----------+-----------+----------+---------+----------+------------+-----------+------------+-------------+------------+---------------+------------+-------------+------------------+---------------+----------+-----------------+-------+---------+-------+-------+------------+----------------+----------------+-----------+---------------+----------------+----------+----------------+---------+---------+------------+----------+----------------+--------------+----------------+--------------+----------------+--------------+-----------+------------------+------------------+------------------+--------------+---------------------+---------------------+---------------------+-------------------------+-------------------------+-------------------------+-------------------------+--------------

In [33]:
full_df.select(
    F.min(
        F.col('Season')
    ).alias('Min Season of Data'),

    F.max(
        F.col('Season')
    ).alias('Max Season of Data'),
).distinct().show()

+------------------+------------------+
|Min Season of Data|Max Season of Data|
+------------------+------------------+
|              1912|              2024|
+------------------+------------------+



## Bring in the Biofile so we can get player names

In [34]:

file_path = f'{os.getcwd()}/data/mlb_data/biofile.csv'
bio_df = spark.read.csv(file_path, header=True, inferSchema=True)

bio_df.show()

+--------+-----------+----------------+--------+----------+------------+-------------+------------------+----------+-------------+---------+------------+-----------+--------------+----------+------------+----------+---------------+-----------+-------------+----+------+------+------+--------------------+---------------+------------+------------+---------+--------------------+--------+-------+----+
|PLAYERID|       LAST|           FIRST|NICKNAME| BIRTHDATE|  BIRTH.CITY|  BIRTH.STATE|     BIRTH.COUNTRY|PLAY.DEBUT|PLAY.LASTGAME|MGR.DEBUT|MGR.LASTGAME|COACH.DEBUT|COACH.LASTGAME| UMP.DEBUT|UMP.LASTGAME| DEATHDATE|     DEATH.CITY|DEATH.STATE|DEATH.COUNTRY|BATS|THROWS|HEIGHT|WEIGHT|            CEMETERY|      CEME.CITY|  CEME.STATE|CEME.COUNTRY|CEME.NOTE|          BIRTH.NAME|NAME.CHG|BAT.CHG| HOF|
+--------+-----------+----------------+--------+----------+------------+-------------+------------------+----------+-------------+---------+------------+-----------+--------------+----------+---------

In [ ]:
pandas_full = pd.DataFrame()
for file in os.listdir(f'{os.getcwd()}/data/mlb_data'):
    file_path = f'{os.getcwd()}/data/mlb_data/{file}'
    print(f'Processing {file_path}')
    df = pd.read_csv(file_path)

    pandas_full = pd.concat([pandas_full, df])


print(pandas_full.head())

Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_1970s_pbp.csv


/tmp/ipykernel_1219/2705840558.py:5: DtypeWarning: Columns (7,50,64,65,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_2000s_pbp.csv


/tmp/ipykernel_1219/2705840558.py:5: DtypeWarning: Columns (63,64,65,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_1960s_pbp.csv


/tmp/ipykernel_1219/2705840558.py:5: DtypeWarning: Columns (7,50,64,65,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processing /home/jdwolfe/Spark-Projects/spark-learning-notebooks/data/mlb_data/full_1950s_pbp.csv


/tmp/ipykernel_1219/2705840558.py:5: DtypeWarning: Columns (7,64,65,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


### ^^ Pandas was taking almost 10 min here to process similar data to pyspark it didnt finish so I skipped it

In [35]:
pitcher_inning_cycles = full_df.groupby('pitcher', 'game id', 'inning').agg(
    F.sum(F.when(F.col('event type') == 20, 1).otherwise(0)).alias('singles'),
    F.sum(F.when(F.col('event type') == 21, 1).otherwise(0)).alias('doubles'),
    F.sum(F.when(F.col('event type') == 22, 1).otherwise(0)).alias('triples'),
    F.sum(F.when(F.col('event type') == 23, 1).otherwise(0)).alias('home runs')
)

pitcher_inning_cycles \
    .filter(
        (F.col('singles') > 0) &
        (F.col('doubles') > 0) &
        (F.col('triples') > 0) &
        (F.col('home runs') > 0)
    ) \
    .groupby('pitcher') \
    .agg(
    F.count('pitcher').alias('count')
).sort('count', ascending=False ) \
    .show()




25/05/11 17:16:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 17:16:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 17:16:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 17:16:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 17:16:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 17:16:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 17:16:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 17:16:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 17:16:03 WARN RowBasedKeyValueBatch: Calling spill() on

+--------+-----+
| pitcher|count|
+--------+-----+
|jacke001|    6|
|briln101|    4|
|lynnl001|    4|
|millk004|    3|
|forsb001|    3|
|hainj101|    3|
|davik002|    3|
|cardd101|    3|
|doakb101|    3|
|paulf001|    3|
|lucar102|    3|
|frieb102|    3|
|hubbc101|    3|
|owenj101|    3|
|hudss101|    3|
|brane101|    3|
|pappm101|    3|
|gullb001|    3|
|wegmb001|    3|
|ruffr101|    3|
+--------+-----+
only showing top 20 rows



## Cycles by Players Leaders

In [11]:
full_df.groupby('batter', 'game id').agg(
    F.sum(F.when(F.col('event type') == 20, 1).otherwise(0)).alias('singles'),
    F.sum(F.when(F.col('event type') == 21, 1).otherwise(0)).alias('doubles'),
    F.sum(F.when(F.col('event type') == 22, 1).otherwise(0)).alias('triples'),
    F.sum(F.when(F.col('event type') == 23, 1).otherwise(0)).alias('home runs')
    ) \
    .filter(

        (F.col('singles') > 0) &
        (F.col('doubles') > 0) &
        (F.col('triples') > 0) &
        (F.col('home runs') > 0)

    ).groupby('batter') \
    .agg(
    F.count('batter').alias('count')
).sort('count', ascending=False ) \
    .show()

25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/11 13:55:35 WARN RowBasedKeyValueBatch: Calling spill() on

+--------+-----+
|  batter|count|
+--------+-----+
|yelic001|    3|
|belta001|    3|
|turnt001|    3|
|gomec002|    2|
|arenn001|    2|
|olerj001|    2|
|boyek101|    2|
|watsb001|    2|
|speic001|    2|
|bretg001|    2|
|hilla001|    2|
|freef001|    2|
|wilkb002|    2|
|fregj101|    2|
|cuddm001|    2|
|whitf001|    2|
|cedec001|    2|
|mcdoo001|    1|
|dejei001|    1|
|teixm001|    1|
+--------+-----+
only showing top 20 rows



## Hit Leaders

In [36]:
full_df.groupby('batter').agg(
    F.sum(F.when(F.col('event type').isin(20,21,22,23), 1).otherwise(0)).alias('total_hits')

).sort('total_hits', ascending=False ).show()

+--------+----------+
|  batter|total_hits|
+--------+----------+
|rosep001|      4256|
|aaroh101|      3698|
|jeted001|      3465|
|yastc101|      3419|
|pujoa001|      3384|
|molip001|      3319|
|musis101|      3317|
|murre001|      3255|
|maysw101|      3238|
|ripkc001|      3184|
|cabrm001|      3174|
|belta001|      3166|
|bretg001|      3154|
|younr001|      3142|
|gwynt001|      3141|
|rodra001|      3115|
|winfd001|      3110|
|suzui001|      3089|
|biggc001|      3060|
|hendr001|      3055|
+--------+----------+
only showing top 20 rows



In [37]:
(full_df
.filter(F.col('Season') >= 2000)
 .groupby('batter').agg(
    F.sum(F.when(F.col('event type').isin(20,21,22,23), 1).otherwise(0)).alias('total_hits')

).sort('total_hits', ascending=False ).show())

+--------+----------+
|  batter|total_hits|
+--------+----------+
|pujoa001|      3384|
|cabrm001|      3174|
|suzui001|      3089|
|belta001|      2976|
|jeted001|      2658|
|canor001|      2639|
|beltc001|      2515|
|rollj001|      2455|
|markn001|      2388|
|ortid001|      2379|
|younm003|      2375|
|huntt001|      2350|
|rodra001|      2324|
|freef001|      2267|
|ramia001|      2234|
|altuj001|      2232|
|pierj002|      2217|
|moliy001|      2168|
|tejam001|      2153|
|martv001|      2153|
+--------+----------+
only showing top 20 rows



In [38]:
(full_df
.filter(F.col('Season') >= 2010)
 .groupby('batter').agg(
    F.sum(F.when(F.col('event type').isin(20,21,22,23), 1).otherwise(0)).alias('total_hits')

).sort('total_hits', ascending=False ).show())

+--------+----------+
|  batter|total_hits|
+--------+----------+
|freef001|      2267|
|altuj001|      2232|
|goldp001|      2056|
|mccua001|      2028|
|andre001|      1963|
|cabrm001|      1954|
|machm001|      1900|
|arenn001|      1826|
|blacc001|      1805|
|vottj001|      1801|
|cruzn002|      1793|
|santc002|      1789|
|canor001|      1764|
|hosme001|      1753|
|martj006|      1741|
|lemad001|      1738|
|casts001|      1722|
|bogax001|      1693|
|markn001|      1684|
|harpb003|      1670|
+--------+----------+
only showing top 20 rows



## Hit Leader for Every Season

In [39]:
window_for_hit_leaders = Window.partitionBy("Season",).orderBy(F.desc("total_hits"))


season_hit_leaders = full_df \
 .groupby('batter', 'Season').agg(
    F.sum(F.when(F.col('event type').isin(20,21,22,23), 1).otherwise(0)).alias('total_hits')
)\
    .withColumn('season_rank', F.rank().over(window_for_hit_leaders)) \
    .filter(F.col('season_rank') == 1) \
    .drop('season_rank')

bio_select   = bio_df.select('PLAYERID', 'NICKNAME', 'LAST')

season_hit_leaders = season_hit_leaders.join(bio_select, bio_select.PLAYERID == season_hit_leaders.batter) \
                    .drop('PLAYERID', 'batter') \
                    .select('NICKNAME', 'LAST', 'Season', 'total_hits')


season_hit_leaders.show(season_hit_leaders.count())

+------------+--------------+------+----------+
|    NICKNAME|          LAST|Season|total_hits|
+------------+--------------+------+----------+
|          Ty|          Cobb|  1912|       226|
|Shoeless Joe|       Jackson|  1912|       226|
|         Max|         Carey|  1913|       171|
|        Tris|       Speaker|  1914|       187|
|          Ty|          Cobb|  1915|       208|
|        Tris|       Speaker|  1916|       196|
|          Ty|          Cobb|  1917|       225|
|      George|         Burns|  1918|       177|
|       Bobby|         Veach|  1919|       189|
|      George|        Sisler|  1920|       257|
|        Jack|         Tobin|  1921|       230|
|      Rogers|       Hornsby|  1921|       230|
|      Rogers|       Hornsby|  1922|       248|
|     Frankie|        Frisch|  1923|       220|
|        Zack|         Wheat|  1924|       209|
|      Rogers|       Hornsby|  1924|       209|
|          Al|       Simmons|  1925|       248|
|      George|         Burns|  1926|    

In [45]:
window_for_hit_leaders = Window.partitionBy("Season",).orderBy(F.desc("total_hits"))


season_hit_leaders = full_df \
 .groupby('batter', 'Season').agg(
    F.sum(F.when(F.col('event type').isin(20,21,22,23), 1).otherwise(0)).alias('total_hits')
)\
    .withColumn('season_rank', F.rank().over(window_for_hit_leaders)) \
    .filter(F.col('season_rank') == 1) \
    .drop('season_rank')

bio_select   = bio_df.select('PLAYERID', 'NICKNAME', 'LAST')

season_hit_leaders = season_hit_leaders \
                    .select( 'Season', 'total_hits', 'batter') \
                    .groupby('batter') \
                    .agg(F.count('batter').alias('Total Hitting Leaders'))

season_hit_leaders = season_hit_leaders \
                    .join(bio_select, bio_select.PLAYERID == season_hit_leaders.batter ) \
                    .drop('PLAYERID', 'batter') \
                    .sort('Total Hitting Leaders', ascending=False )




season_hit_leaders.show(season_hit_leaders.count())

+---------------------+------------+--------------+
|Total Hitting Leaders|    NICKNAME|          LAST|
+---------------------+------------+--------------+
|                    7|      Ichiro|        Suzuki|
|                    7|        Pete|          Rose|
|                    5|        Tony|         Gwynn|
|                    3|       Kirby|       Puckett|
|                    3|      Rogers|       Hornsby|
|                    3|          Ty|          Cobb|
|                    2|      Richie|       Ashburn|
|                    2|        Trea|        Turner|
|                    2|       Frank|     McCormick|
|                    2|        Jose|        Altuve|
|                    2|      George|         Brett|
|                    2|      George|         Burns|
|                    2|        Vada|        Pinson|
|                    2|        Dale|      Mitchell|
|                    2|        Paul|       Molitor|
|                    2|        Stan|        Musial|
|           